In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [3]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [6]:
df.reset_index(drop= True).head()

,index,order_id,product_id,add_to_cart_order,reordered,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,tip,product_name,aisle_id,department_id,department,aisle,county
0,5006171,1,13176,6,0,112108,4,4,10,9.0,1,Bag of Organic Bananas,24,4,produce,fresh fruits,Inyo
1,5006176,1,11109,2,1,112108,4,4,10,9.0,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,Inyo
2,5006175,1,10246,3,0,112108,4,4,10,9.0,1,Organic Celery Hearts,83,4,produce,fresh vegetables,Inyo
3,5006174,1,22035,8,1,112108,4,4,10,9.0,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,Inyo
4,5006173,1,49683,4,0,112108,4,4,10,9.0,1,Cucumber Kirby,83,4,produce,fresh vegetables,Inyo


### **P2:** Lassen sich regionale Unterschiede im Trinkgeldverhalten erkennen?

In [7]:
df_county_tip1 = df.groupby(["county", "order_id"]).agg(tip = ("tip", "mean")).reset_index()

In [11]:
df_county_tip2 = df_county_tip1.groupby("county").agg(tip = ("tip", "mean"), orders= ("order_id", "count")).sort_values(by= "tip", ascending= False).reset_index()

In [13]:
df_county_tip2.head()

,county,tip,orders
0,San Diego,0.530895,2379
1,Santa Barbara,0.507548,13977
2,Lassen,0.502494,2406
3,Del Norte,0.501239,2017
4,Modoc,0.492131,9467


In [17]:
px.bar(df_county_tip2, x= "county", y= "tip")

In [126]:
# hline bei 0.5 
# schlechtestes County hervorheben mit Zahl (siehe bestes)
# vllt hintergrundfarbe entfernen

fig1 = go.Figure()

for county in df_county_tip2.county:
 fig1.add_trace(
  go.Bar(
   x= df_county_tip2.loc[df_county_tip2.county == county].county,
   y= df_county_tip2.loc[df_county_tip2.county == county].tip,
   name= county,
   showlegend= True,
   marker= dict(color= px.colors.qualitative.Set1[8]),
   width= 0.8
  )
 )

 
fig1.update_traces(marker_color= px.colors.qualitative.G10[8], text= df_county_tip2.loc[df_county_tip2.tip == df_county_tip2.tip.max()].tip[0].round(3), selector= dict(name= df_county_tip2.loc[df_county_tip2.tip == df_county_tip2.tip.max()].county[0]))

fig1.update_layout(legend=dict(groupclick="toggleitem"),title= dict(text= "<b>Regionale Unterschiede im Trinkgeldverhalten</b>", font_size= 20, xanchor= "center", yanchor= "top", y=0.9, x=0.43), legend_title_text = "<b>Counties</b>", height= 550)
fig1.update_xaxes(title_text="<b>Counties</b>")
fig1.update_yaxes(title_text="<b>Tipp Wshk.</b>")
fig1.show()

#### old

In [ ]:
df_county_tip = df.groupby("county").agg(tip = ("tip", "sum"), tip_opp = ("tip", "count")).reset_index()
df_county_tip["no_tip"] = df_county_tip.tip_opp - df_county_tip.tip

In [ ]:
df_county_tip["tip_norm"], df_county_tip["no_tip_norm"] = [df_county_tip.iloc[:, 1:2].apply(lambda x: x / df_county_tip.tip_opp, axis= 0), df_county_tip.iloc[:, 3:4].apply(lambda x: x / df_county_tip.tip_opp, axis= 0)]

In [ ]:
df_county_tip.drop(["tip_opp"], axis= 1, inplace= True)

In [ ]:
df_county_tip.head()

,county,tip,no_tip,tip_norm,no_tip_norm
0,Alameda,7071,9845,0.418007,0.581993
1,Alpine,59712,73571,0.448009,0.551991
2,Amador,55238,65874,0.456090,0.543910
3,Calaveras,229220,273259,0.456178,0.543822
4,Colusa,16970,17973,0.485648,0.514352


In [ ]:
df_county_tip.loc[df_county_tip.tip_norm > df_county_tip.no_tip_norm]

,county,tip,no_tip,tip_norm,no_tip_norm
6,Del Norte,10075,9235,0.521750,0.478250
7,El Dorado,8183,7240,0.530571,0.469429
10,Humboldt,3321,3080,0.518825,0.481175
16,Lassen,11678,11522,0.503362,0.496638
23,Modoc,50644,44438,0.532635,0.467365
24,Mono,15075,14790,0.504771,0.495229
34,San Bernardino,75,73,0.506757,0.493243
35,San Diego,13636,10684,0.560691,0.439309
40,Santa Barbara,93390,80136,0.538190,0.461810
53,Tuolumne,16847,15186,0.525926,0.474074


In [ ]:
pd.crosstab(df.county, df.tip, margins= True, normalize= False).head()

tip,0,1,All
county,,,
Alameda,9845,7071,16916
Alpine,73571,59712,133283
Amador,65874,55238,121112
Calaveras,273259,229220,502479
Colusa,17973,16970,34943


In [ ]:
df_county_tip_sorted = df_county_tip.sort_values(by= ["tip_norm"], ascending= False)
px.bar(df_county_tip_sorted, x= "county", y= "tip_norm")